### Example for retrieving feature store records

Assumptions
- You will need the Feature Group name that is provisioned and the feature id
- Create a lambda function (used 1 GB memory for this example) and its related role has access to SageMaker feature store
- Notebook role has access to invoke lambda
- Optional - [Provisioned Concurrrency](https://aws.amazon.com/blogs/aws/new-provisioned-concurrency-for-lambda-functions/) configured based on your use-case


In [ ]:
import boto3
import json
import numpy as np
import time

In [ ]:
def get_stats(times):
    a = np.array(times)
    p_50 = np.percentile(a, 50)
    p_95 = np.percentile(a, 95)
    p_99 = np.percentile(a, 99)
    avg = np.average(a)
    
    return {
        'avg':avg, 
        'p_50': p_50, 
        'p_95': p_95, 
        'p_99': p_99}

In [ ]:
def invoke_lambda(client, fg_name, feat_id):   
    payload={
      "featureGrpName": fg_name,
      "featureId": str(feat_id)
    }

    response = client.invoke(
            FunctionName="lambda_featurestore",
            InvocationType='RequestResponse',
            Payload=json.dumps(payload)
        )
    
    return json.loads(response['Payload'].read())

In [ ]:
fg_name = 'ingest-fg-06-17-2021-14-46-44'
feat_id = 1
client = boto3.client('lambda')
times = []
for i in range(10):
    ret = invoke_lambda(client, fg_name, feat_id)
    times.append(ret)

In [ ]:
print ('fs_read1', get_stats([d['fs_read1'] for d in times]))

In [ ]:
print ('fs_read1', get_stats([d['fs_read2'] for d in times]))